In [1]:
import json
from datasets import load_from_disk
from sentence_transformers import SentenceTransformer, CrossEncoder, util
import torch

In [2]:
with open('../data/preprocess_wiki.json', 'r', encoding='utf-8') as f:
    wiki = json.load(f)
wiki['0']

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다. 이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.   첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.   두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다. 두 목록은 모두 가나다 순이다. 일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [3]:
corpus = [wiki[str(i)]['text'] for i in range(len(wiki))]

In [4]:
len(corpus)

60613

In [ ]:
# cross_encoder = CrossEncoder('outputs/torch_versionup/checkpoint-666')
# cross_encoder = CrossEncoder('klue/bert-base')
cross_encoder = CrossEncoder('amberoad/bert-multilingual-passage-reranking-msmarco')
# cross_encoder = CrossEncoder('bespin-global/klue-korsts-roberta-base-sentence-embedding')


In [5]:
# bi_encoder = SentenceTransformer('KR-SBERT/KR-SBERT-V40K-klueNLI-augSTS')
# bi_encoder = SentenceTransformer('bespin-global/klue-korsts-roberta-base-sentence-embedding')
bi_encoder = SentenceTransformer('Huffon/sentence-klue-roberta-base')
bi_encoder.max_seq_length = 384

corpus_embeddings = bi_encoder.encode(corpus, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/1895 [00:00<?, ?it/s]

In [14]:
# from rank_bm25 import BM25Okapi
# from sklearn.feature_extraction import _stop_words
# import string
from tqdm.autonotebook import tqdm
import numpy as np

# bm25 = BM25Okapi(corpus)

In [16]:
import pickle

with open('/opt/ml/elastic_valid_500.bin','rb') as f:
    elastic_valid = pickle.load(f)

In [9]:
def search(query, k):
    # bm25_retrieval, bi_encoder_retrieval, cross_encoder_retrieval = [], [], []
    bi_encoder_retrieval, cross_encoder_retrieval = [], []
    # # print("Input question:", query)

    # ##### BM25 search (lexical search) #####
    # bm25_scores = bm25.get_scores(query)
    # top_n = np.argpartition(bm25_scores, -5)[:]
    # bm25_hits = [{'corpus_id': idx, 'score': bm25_scores[idx]} for idx in top_n]
    # bm25_hits = sorted(bm25_hits, key=lambda x: x['score'], reverse=True)
    
    # # print("Top-3 lexical search (BM25) hits")
    # # for hit in bm25_hits[0:3]:
    # #     # print("\t{:.3f}\t{}".format(hit['score'], corpus[hit['corpus_id']].replace("\n", " ")))
    # #     print("\t{:.3f}\t{}".format(hit['score'], hit['corpus_id']))
    # for hit in bm25_hits[0:k]:
    #     bm25_retrieval.append(hit['corpus_id'])

    ##### Sematic Search #####
    # Encode the query using the bi-encoder and find potentially relevant passages
    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    question_embedding = question_embedding.cuda()
    hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=100)
    hits = hits[0]  # Get the hits for the first query

    ##### Re-Ranking #####
    # Now, score all retrieved passages with the cross_encoder
    cross_inp = [[query, corpus[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    # Output of top-5 hits from bi-encoder
    # print("\n-------------------------\n")
    # print("Top-3 Bi-Encoder Retrieval hits")
    hits = sorted(hits, key=lambda x: x['score'], reverse=True)
    for hit in hits[0:k]:
        bi_encoder_retrieval.append(hit['corpus_id'])
    # for hit in hits[0:3]:
    #     # print("\t{:.3f}\t{}".format(hit['score'], corpus[hit['corpus_id']].replace("\n", " ")))
    #     print("\t{:.3f}\t{}".format(hit['cross-score'], hit['corpus_id']))

    # Output of top-5 hits from re-ranker
    # print("\n-------------------------\n")
    # print("Top-3 Cross-Encoder Re-ranker hits")
    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)
    for hit in hits[0:k]:
        cross_encoder_retrieval.append(hit['corpus_id'])
    # for hit in hits[0:3]:
    #     # print("\t{:.3f}\t{}".format(hit['cross-score'], corpus[hit['corpus_id']].replace("\n", " ")))
    #     print("\t{:.3f}\t{}".format(hit['cross-score'], hit['corpus_id']))
    return bi_encoder_retrieval, cross_encoder_retrieval

In [10]:
td = load_from_disk('../data/new_train_dataset')['train']

In [11]:
a, b = search(td[0]['question'], 5)
len(a)


5

In [12]:
origin_vd = load_from_disk('/opt/ml/data/train_dataset')['validation']
len(origin_vd['document_id'])

240

In [49]:
question = origin_vd['question']
question_embedding = bi_encoder.encode(question, convert_to_tensor=True)
question_embedding = question_embedding.cuda()
hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=500)

st_v = {}
for i, q in enumerate(question):
    st_v[q] = hits[i]

# hits[0][0]

In [53]:
with open('sentence_transformer_valid.pickle', 'wb') as f:
    pickle.dump(st_v, f, pickle.HIGHEST_PROTOCOL)

In [62]:
test_dataset = load_from_disk('../data/test_dataset')['validation']

question = test_dataset['question']
question_embedding = bi_encoder.encode(question, convert_to_tensor=True)
question_embedding = question_embedding.cuda()
hits = util.semantic_search(question_embedding, corpus_embeddings, top_k=500)

st_t = {}
for i, q in enumerate(question):
    st_t[q] = hits[i]

In [66]:
with open('sentence_transformer_test.pickle', 'wb') as f:
    pickle.dump(st_t, f, pickle.HIGHEST_PROTOCOL)

In [65]:
len(st_v[list(st_v.keys())[0]])

500

In [63]:
len(st_t[list(st_t.keys())[0]])

500

In [19]:
valid_dataset = load_from_disk("/opt/ml/data/new_train_dataset/validation")
query = valid_dataset['question']
context = valid_dataset['context']

# top_k_list = [1,5,10,15,20,25,30]
top_k_list = [20]

for top_k in top_k_list:
    # elastic_acc = 0
    # hybrid_acc = 0
    # bm25_acc = 0
    es_acc = 0
    bi_encoder_acc = 0
    cross_encoder_acc = 0

    for i in tqdm(range(len(query))):
        q = query[i]
        ground_truth = origin_vd[i]['document_id']

        bi_encoder_top_k, cross_encoder_top_k = search(q, top_k)
        # bm25_top_k, bi_encoder_top_k, cross_encoder_top_k = search(q, top_k)

        # if ground_truth in bm25_top_k:
        #     bm25_acc += 1
        es_top_k = []
        for j in range(top_k):
            es_top_k.append(elastic_valid[q][j])

        if ground_truth in es_top_k:
            es_acc += 1
        if ground_truth in bi_encoder_top_k:
            bi_encoder_acc += 1
        if ground_truth in cross_encoder_top_k:
            cross_encoder_acc += 1

    print('score_top_k : ', top_k)
    # print('bm25 ACC : ', bm25_acc / len(query))
    print('es ACC : ', es_acc / len(query))
    print('bi-encoder ACC :',bi_encoder_acc / len(query))
    print('cross-encoder ACC :',cross_encoder_acc / len(query))

    print()

  0%|          | 0/240 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


RuntimeError: The size of tensor a (881) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
KR-SBERT + klue/bert-base
score_top_k :  20
es ACC :  0.9375
bi-encoder ACC : 0.37916666666666665
cross-encoder ACC : 0.37916666666666665


bespin-global/klue-korsts-roberta-base-sentence-embedding + klue/bert-base
score_top_k :  20
es ACC :  0.9375
bi-encoder ACC : 0.5291666666666667
cross-encoder ACC : 0.5291666666666667

Huffon/sentence-klue-roberta-base + klue/bert-base
score_top_k :  20
es ACC :  0.9375
bi-encoder ACC : 0.5666666666666667
cross-encoder ACC : 0.5666666666666667
